# Reading Data

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("C:\\Users\\byunh\\Deep Learning Tensorflow\\", one_hot=True)

C:\Users\byunh\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting C:\Users\byunh\Deep Learning Tensorflow\train-images-idx3-ubyte.gz
Extracting C:\Users\byunh\Deep Learning Tensorflow\train-labels-idx1-ubyte.gz
Extracting C:\Users\byunh\Deep Learning Tensorflow\t10k-images-idx3-ubyte.gz
Extracting C:\Users\byunh\Deep Learning Tensorflow\t10k-labels-idx1-ubyte.gz


In [2]:
print(mnist.train.num_examples) # Number of training data
print(mnist.test.num_examples) # Number of test data

55000
10000


# Setting hyper-parameters

In [ ]:
# architecture hyper-parameter
noofdatapoints = mnist.train.num_examples

learningrate = 0.001
nepochs = 10
batch_size = 512
noofbatches = noofdatapoints//batch_size
print("no of batches =", noofbatches)

n_input = 784 # 28x28 image
n_classes = 10 # 1 for each digit [0-9]
dropout = 0.75 

# Defining x and y as placeholders

In [4]:
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)
print(X.shape, Y.shape)

(?, 784) (?, 10)


# Defining model using conv2d, Relu and Maxpooling

In [5]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [7]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [8]:
def conv_net(x, weights, biases, dropout):
    # reshape input to 28x28 size
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max pooling
    conv1 = maxpool2d(conv1, k=2)

    # Convolution layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max pooling
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# Initializing Weights and Biases

In [9]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Define cost, optimizer, accuracy and save weight

In [10]:
# Create the model
model = conv_net(X, weights, biases, keep_prob)
print(model)
# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
train_min = tf.train.AdamOptimizer(learning_rate=learningrate).minimize(loss)

# Evaluate model
correct_model = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_model, tf.float32))
#save weight
weights_saver = tf.train.Saver(var_list=weights)
# Initializing the variables
init = tf.global_variables_initializer()

Tensor("Add_1:0", shape=(?, 10), dtype=float32)


# Running the session

In [11]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(nepochs):
        for _ in range(noofbatches):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
           
            # Use training data for optimization
            sess.run(train_min, feed_dict={X:batch_x, Y:batch_y, keep_prob: dropout})
        # Validate after every epoch
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        losscalc, accuracycalc = sess.run([loss, accuracy], 
                                          feed_dict={X:batch_x, Y:batch_y, keep_prob: 1.0})
        print("Epoch: %d, Loss: %0.4f, Accuracy: %0.4f"%(epoch, losscalc, accuracycalc))
            
    # When the training is complete and you are happy with the result
    accuracycalc = sess.run(accuracy, 
                            feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0})
    print("Testing accuracy: %0.4f"%(accuracycalc))
    weights_saver.save(sess,'C:\\Users\\byunh\\Deep Learning Tensorflow\\model.ckpt')
sess.close()

Epoch: 0, Loss: 1628.4896, Accuracy: 0.9141
Epoch: 1, Loss: 838.2557, Accuracy: 0.9531
Epoch: 2, Loss: 543.3802, Accuracy: 0.9570
Epoch: 3, Loss: 699.4945, Accuracy: 0.9473
Epoch: 4, Loss: 304.4406, Accuracy: 0.9648
Epoch: 5, Loss: 126.7414, Accuracy: 0.9785
Epoch: 6, Loss: 106.2631, Accuracy: 0.9824
Epoch: 7, Loss: 146.0192, Accuracy: 0.9785
Epoch: 8, Loss: 36.1198, Accuracy: 0.9922
Epoch: 9, Loss: 107.0914, Accuracy: 0.9844
Testing accuracy: 0.9765


In [ ]:
# use saved weights to train  train Fashion MNIST data set 

# fashion mnist


In [14]:
data = input_data.read_data_sets('data/fashion', one_hot=True)


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/fasion\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/fasion\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/fasion\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/fasion\t10k-labels-idx1-ubyte.gz


# Reset noofdatapoints

In [17]:
noofdatapoints = data.train.num_examples

# Running the session


In [18]:
with tf.Session() as sess:
    sess.run(init)
    
    weights_saver.restore(sess,'C:\\Users\\byunh\\Deep Learning Tensorflow\\model.ckpt')
    
    for epoch in range(nepochs):
        for _ in range(noofbatches):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            # Use training data for optimization
            sess.run(train_min, feed_dict={X:batch_x, Y:batch_y, keep_prob: dropout})
        # Validate after every epoch
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        losscalc, accuracycalc = sess.run([loss, accuracy], 
                                          feed_dict={X:batch_x, Y:batch_y, keep_prob: 1.0})
        print("Epoch: %d, Loss: %0.4f, Accuracy: %0.4f"%(epoch, losscalc, accuracycalc))
            
    # When the training is complete and you are happy with the result
    accuracycalc = sess.run(accuracy, 
                            feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0})
    print("Testing accuracy: %0.4f"%(accuracycalc))
    

INFO:tensorflow:Restoring parameters from C:\Users\byunh\Deep Learning Tensorflow\model.ckpt
Model restored.
Epoch: 0, Loss: 498.7940, Accuracy: 0.9512
Epoch: 1, Loss: 425.2921, Accuracy: 0.9551
Epoch: 2, Loss: 212.6940, Accuracy: 0.9785
Epoch: 3, Loss: 238.0041, Accuracy: 0.9746
Epoch: 4, Loss: 284.1601, Accuracy: 0.9648
Epoch: 5, Loss: 173.4926, Accuracy: 0.9746
Epoch: 6, Loss: 315.9897, Accuracy: 0.9648
Epoch: 7, Loss: 184.8718, Accuracy: 0.9668
Epoch: 8, Loss: 119.0837, Accuracy: 0.9785
Epoch: 9, Loss: 279.4896, Accuracy: 0.9629
Testing accuracy: 0.9745
